In [ ]:
#---#| default_exp utils

# Utils

In [ ]:
from peptdeep.utils import *

### Supported devices in peptdeep

In [ ]:
get_device('cuda')

(device(type='cpu'), 'cpu')

In [ ]:
torch_devices

{'gpu': {'is_available': <function torch.cuda.is_available() -> bool>,
  'device': 'cuda'},
 'cuda': {'is_available': <function torch.cuda.is_available() -> bool>,
  'device': 'cuda'},
 'mps': {'is_available': <function peptdeep.utils._is_mps_available() -> bool>,
  'device': 'mps'},
 'm1': {'is_available': <function peptdeep.utils._is_mps_available() -> bool>,
  'device': 'mps'}}

## Plotting functions

In [ ]:
from peptdeep.utils import parse_ms_file_names_to_dict

d = parse_ms_file_names_to_dict([
    'a.raw', 'b.ms_data.hdf','c_HCDFT.mgf'
])
assert 'a' in d
assert 'b' in d
assert 'c' in d